In [1]:
import pandas as pd
import altair as alt
import numpy as np

In [59]:
cdf = pd.read_csv('/Users/curtislisle/Dropbox/ipython-notebooks/D3M/TERRA/s4_full_height_leaf_day_v2.csv')

Lets try to generate an interactive plot of the canopy height at any given day using this dataframe.  Use ipython widgets to allow the day to be selected.  This may be too slow, considering we are doing a full-field rendering, but lets try it first from PANDAS, then from an array if speed is necessary. It isn't enough to just select a day (as seen below) because all plants aren't measured on every single day.  

In [24]:
selectedDay = 54

day_df = cdf.loc[cdf['day_offset'] == selectedDay]
print(len(day_df),"measurements were made on day",selectedDay)

260 measurements were made on day 54


In [25]:
charttitle = 'Season4 Canopy Height: Day '+str(selectedDay)
alt.Chart(day_df,title=charttitle).mark_rect().encode(
    x='column:O',
    y='range:O',
    color='canopy_height',
    tooltip=[
        alt.Tooltip('cultivar', title='Cultivar'),
        alt.Tooltip('canopy_height:Q', title='Canopy Height'),
        alt.Tooltip('range:O',title='range'),
        alt.Tooltip('column:O',title='column')
    ]
)

alt.Chart(...)

We could use PANDAS to do find the max of any canopy_height up until this day.  This works only if canopy_height never decreases (monotonically increasing), which isn't true, so we really want to find the "nearest" measurement at or before the query day. This can be done as a multi-step query:  (1) subselect to measurements <= to the query day, (2) find the max day_offset, (3) query from that max day_offset (the most recent reading). 

In [99]:
selectedDay = 150
# first get rid of observations after the query day
before_df = cdf.loc[cdf['day_offset'] <= selectedDay]
print(before_df.shape)

# group all the measurements so far by cultivar 
grouped = before_df.groupby(['cultivar'])

# now loop through these by cultivar and select only the measurement with the highest day_offset value (the most recent)
recentlist = []
for name, group in grouped:
    #print(name)
    selected = group['day_offset'].idxmax()  # this selects the highest value index
    # the index is a lookup into the original dataframe, so put this entry in the list for plotting
    recentlist.append(cdf.iloc[selected])  
    
# how many cultivars did we find that had a measurement on or before our day?    
print(len(recentlist),"cultivars have been measured on or before day",selectedDay)
recent_df = pd.DataFrame(recentlist)
recent_df.iloc[0]

(9441, 13)
351 cultivars have been measured on or before day 150


canopy_height                              364
column                                      14
cultivar                            Big_Kahuna
date                       2017-08-29 12:00:00
leaf_angle_alpha                       2.18209
leaf_angle_beta                        1.58035
leaf_angle_chi                         1.81331
leaf_angle_mean                       0.439688
range                                       54
season                                       4
day_offset                                 120
single_xgboost                         361.378
abserror_single_xgboost               0.720449
Name: 4414, dtype: object

In [104]:
charttitle = 'Season4 Canopy Height: Day '+str(selectedDay)
alt.Chart(recent_df,title=charttitle).mark_rect().encode(
    x='column:O',
    y='range:O',
    color='canopy_height',
    tooltip=[
        alt.Tooltip('cultivar', title='Cultivar'),
        alt.Tooltip('canopy_height:Q', title='Canopy Height'),
        alt.Tooltip('range:O',title='range'),
        alt.Tooltip('column:O',title='column')
    ]
)

alt.Chart(...)

Now we have a dataframe "recent_df" which has the most recent measurement for each cultivar.  We aren't guaranteed that all cultivars are present, it depends on when the measurements came in.  This is not a complete coverage of the field, it is only what measurements have been made so far (one per cultivar), So first lets figure out how to get the canopy height out of this for a particular cultivar or for a particular location in the field:

In [93]:
recent_df.loc[recent_df['cultivar'] =='Big_Kahuna']

,canopy_height,column,cultivar,date,leaf_angle_alpha,leaf_angle_beta,leaf_angle_chi,leaf_angle_mean,range,season,day_offset,single_xgboost,abserror_single_xgboost
4414,364.0,14,Big_Kahuna,2017-08-29 12:00:00,2.182092,1.580347,1.813309,0.439688,54,4,120.0,361.377565,0.720449


In [101]:
recent_df.loc[(recent_df['column'] ==14) & (recent_df['range'] ==54)]

,canopy_height,column,cultivar,date,leaf_angle_alpha,leaf_angle_beta,leaf_angle_chi,leaf_angle_mean,range,season,day_offset,single_xgboost,abserror_single_xgboost
4414,364.0,14,Big_Kahuna,2017-08-29 12:00:00,2.182092,1.580347,1.813309,0.439688,54,4,120.0,361.377565,0.720449


In [94]:
recent_df.loc[recent_df['cultivar'] =='Big_Kahuna']['canopy_height'].values[0]

364.0

Now we know how to find entries in our "most recent measurement" dataframe.  Lets fill out the field with all the measurements taken so far.  We can do this by Cultivar or by location.  By cultivar isn't exactly right because the same cultivar is planted in more than one location, we want to find the height of a location, of a cultivar. 

In [92]:
maxColumn = cdf.describe().loc['max','column']
maxRange = cdf.describe().loc['max','range']
print('max range:',maxRange, 'max column:',maxColumn)

max range: 54.0 max column: 16.0


In [95]:
def addPlotMarker(cultivar,rng,column,height):
    global plotlist
    mark = {}
    mark['cultivar'] = cultivar
    mark['range'] = rng
    mark['column'] = column
    mark['canopy_height'] = height
    plotlist.append(mark)

Here is an algorithm that plots by Cultivar, which isn't quite right, but the result looks nice...

In [96]:
import numpy as np
plotlist = []

for rng in range(2,int(maxRange)):
    for col in range(2,int(maxColumn)):
        #print(rng,col)
        # find which cultivar is in this spot in the field
        CultivarListInThisSpot = cdf.loc[(cdf['range'] == rng) & (cdf['column']==col)]['cultivar']
        # return a Series of the cultivar names. If the square isn't empty, get the cultivar name from the list. 
        # all cultivar names should be identical since we have selected multiple measurements (on different days) from the same location
        if len(CultivarListInThisSpot)> 0:
            thisCultivar = CultivarListInThisSpot.values[0]
            thisMeasurement = recent_df.loc[recent_df['cultivar'] == thisCultivar]['canopy_height']
            # depending on the day, we might or might not have had a previous measurement, so check there was a measure before plotting
            if len(thisMeasurement)>0:
                thisMeasurementValue = recent_df.loc[recent_df['cultivar'] == thisCultivar]['canopy_height'].values[0]
                addPlotMarker(thisCultivar,rng,col,thisMeasurementValue)
plotdf = pd.DataFrame(plotlist)
print('plotted',len(plotlist),'values')
plotdf.head()
#len(plotdf)

plotted 703 values


,cultivar,range,column,canopy_height
0,Big_Kahuna,2,14,364.0
1,SP1615,2,15,388.0
2,PI329465,3,2,317.0
3,PI22913,3,3,312.0
4,PI145626,3,4,326.0


In [97]:
charttitle = 'Season4 Canopy Height: Day '+str(selectedDay)
alt.Chart(plotdf,title=charttitle).mark_rect().encode(
    x='column:O',
    y='range:O',
    color='canopy_height',
    tooltip=[
        alt.Tooltip('cultivar', title='Cultivar'),
        alt.Tooltip('canopy_height:Q', title='Canopy Height'),
        alt.Tooltip('range:O',title='range'),
        alt.Tooltip('column:O',title='column')
    ]
)

alt.Chart(...)

Now lets modify this algorithm to plot values by the location, instead of looking up by cultivar

In [108]:
selectedDay = 150
# first get rid of observations after the query day
before_df = cdf.loc[cdf['day_offset'] <= selectedDay]
print(before_df.shape)

# group all the measurements so far by cultivar 
grouped = before_df.groupby(['range','column'])

# now loop through these by cultivar and select only the measurement with the highest day_offset value (the most recent)
recentlist = []
for name, group in grouped:
    #print(name)
    selected = group['day_offset'].idxmax()  # this selects the highest value index
    # the index is a lookup into the original dataframe, so put this entry in the list for plotting
    recentlist.append(cdf.iloc[selected])  
    
# how many cultivars did we find that had a measurement on or before our day?    
print(len(recentlist),"cultivars have been measured on or before day",selectedDay)
recent_df = pd.DataFrame(recentlist)
recent_df.iloc[0]

(9441, 13)
727 cultivars have been measured on or before day 150


canopy_height                              328
column                                      14
cultivar                            Big_Kahuna
date                       2017-08-10 12:00:00
leaf_angle_alpha                       1.93273
leaf_angle_beta                        1.42517
leaf_angle_chi                         1.87447
leaf_angle_mean                       0.437473
range                                        2
season                                       4
day_offset                                 101
single_xgboost                         329.553
abserror_single_xgboost               0.473389
Name: 5558, dtype: object

In [109]:
import numpy as np
plotlist = []

cultivarCount = 0
measurementCount = 0
for rng in range(2,int(maxRange)):
    for col in range(2,int(maxColumn)):
        #print(rng,col)
        # find which cultivar is in this spot in the field
        CultivarListInThisSpot = cdf.loc[(cdf['range'] == rng) & (cdf['column']==col)]['cultivar']
        # return a Series of the cultivar names. If the square isn't empty, get the cultivar name from the list. 
        # all cultivar names should be identical since we have selected multiple measurements (on different days) from the same location
        if len(CultivarListInThisSpot)> 0:
            cultivarCount += 1
            thisCultivar = CultivarListInThisSpot.values[0]
            thisMeasurement = recent_df.loc[(recent_df['range'] == rng) & (recent_df['column'] == col)]['canopy_height']
            # depending on the day, we might or might not have had a previous measurement, so check there was a measure before plotting
            if len(thisMeasurement)>0:
                measurementCount += 1
                thisMeasurementValue = thisMeasurement.values[0]
                addPlotMarker(thisCultivar,rng,col,thisMeasurementValue)
plotdf = pd.DataFrame(plotlist)

print('cultivars found:',cultivarCount)
print('measurements found:',measurementCount)
print('plotted',len(plotlist),'values')
plotdf.head()

cultivars found: 703
measurements found: 703
plotted 703 values


,cultivar,range,column,canopy_height
0,Big_Kahuna,2,14,328.0
1,SP1615,2,15,316.0
2,PI329465,3,2,317.0
3,PI22913,3,3,307.0
4,PI145626,3,4,326.0


In [110]:
charttitle = 'Season4 Canopy Height: Day '+str(selectedDay)
alt.Chart(plotdf,title=charttitle).mark_rect().encode(
    x='column:O',
    y='range:O',
    color='canopy_height',
    tooltip=[
        alt.Tooltip('cultivar', title='Cultivar'),
        alt.Tooltip('canopy_height:Q', title='Canopy Height'),
        alt.Tooltip('range:O',title='range'),
        alt.Tooltip('column:O',title='column')
    ]
)

alt.Chart(...)

In [111]:
plotdf.loc[plotdf['cultivar']=='SP1615']

,cultivar,range,column,canopy_height
1,SP1615,2,15,316.0
699,SP1615,53,3,143.0
701,SP1615,53,9,342.0
702,SP1615,53,13,318.0


### Standalone Execution of an interactive rendering showing canopy height for any day picked

Below are the supporting algorithms and the calling *main routine* for a standalone rendering algorithm to show the cultivar height of the whole field at any point in time. This approach to filtering the results should be extendable to show other mid-season measurements as well. 

In [1]:

def addPlotMarker(cultivar,rng,column,height):
    global plotlist
    mark = {}
    mark['cultivar'] = cultivar
    mark['range'] = rng
    mark['column'] = column
    mark['canopy_height'] = height
    plotlist.append(mark)

# this method takes an input day of the season and generates an output dataframe with the most recent
# canopy height measurement taken for each location in the field.  It is a way to watch the field develop
# over time during the season. 

def renderCanopyHeightOnDay(selectedDay):
    global cdf
    global maxRange
    global maxColumn
    
    # first get rid of observations after the query day
    before_df = cdf.loc[cdf['day_offset'] <= selectedDay]
    print(before_df.shape)

    # group all the measurements so far by cultivar 
    grouped = before_df.groupby(['range','column'])

    # now loop through these by cultivar and select only the measurement with the highest day_offset value (the most recent)
    recentlist = []
    for name, group in grouped:
        #print(name)
        selected = group['day_offset'].idxmax()  # this selects the highest value index
        # the index is a lookup into the original dataframe, so put this entry in the list for plotting
        recentlist.append(cdf.iloc[selected])  

    # how many cultivars did we find that had a measurement on or before our day?    
    print(len(recentlist),"cultivars have been measured on or before day",selectedDay)
    recent_df = pd.DataFrame(recentlist)
    
    # now fill out the entire field by querying the values at each location from the 
    # recent dataframe and filling in a plotting list.  This global list (plotlist) needs to be empty
    # before running this algorithm.  
    
    # TODO: remove global dependency/ side effect on plotlist, maxColumn, maxRange?
   
    cultivarCount = 0
    measurementCount = 0
    # go once across the entire field by using range and column indices
    for rng in range(2,int(maxRange)):
        for col in range(2,int(maxColumn)):
            #print(rng,col)
            # find which cultivar is in this spot in the field
            CultivarListInThisSpot = cdf.loc[(cdf['range'] == rng) & (cdf['column']==col)]['cultivar']
            # return a Series of the cultivar names. If the square isn't empty, get the cultivar name from the list. 
            # all cultivar names should be identical since we have selected multiple measurements (on different days) from the same location
            if len(CultivarListInThisSpot)> 0:
                cultivarCount += 1
                thisCultivar = CultivarListInThisSpot.values[0]
                thisMeasurement = recent_df.loc[(recent_df['range'] == rng) & (recent_df['column'] == col)]['canopy_height']
                # depending on the day, we might or might not have had a previous measurement, so check there was a measurement
                # before plotting.  This filter prevents a run-time error trying to plot non-existent measurements.  See the
                # "else" case below for when there is no previous measurement. 
                if len(thisMeasurement)>0:
                    measurementCount += 1
                    thisMeasurementValue = thisMeasurement.values[0]
                    addPlotMarker(thisCultivar,rng,col,thisMeasurementValue)
                else:
                    # fill in empty entries for locations where there were no measurements. This happens more during
                    # the early part of the season because measurements haven't been taken in some locations yet. This
                    # way, the plot will always render the full field because all locations will have an entry, even
                    # if it is zero because no measurements have been taken yet.
                    addPlotMarker(thisCultivar,rng,col,0.0)
             
    plotdf = pd.DataFrame(plotlist)
    print('cultivars found:',cultivarCount)
    print('measurements found:',measurementCount)
    print('plotted',len(plotlist),'values')
    return plotdf


In [2]:
import pandas as pd
import numpy as np
cdf = pd.read_csv('/Users/curtislisle/Dropbox/ipython-notebooks/D3M/TERRA/s4_full_height_leaf_day.csv')

In [4]:
import altair as alt

maxColumn = cdf.describe().loc['max','column']
maxRange = cdf.describe().loc['max','range']

# this accumulates, so should be cleared out before each rendering
plotlist = []
selectedDay = 5
plotdf = renderCanopyHeightOnDay(selectedDay)

charttitle = 'Season4 Canopy Height: Day '+str(selectedDay)
alt.Chart(plotdf,title=charttitle).mark_rect().encode(
x='column:O',
y='range:O',
color='canopy_height:Q',
tooltip=[
    alt.Tooltip('cultivar:O', title='Cultivar'),
    alt.Tooltip('canopy_height:Q', title='Canopy Height'),
    alt.Tooltip('range:O',title='range'),
    alt.Tooltip('column:O',title='column')
]
)

(0, 11)
0 cultivars have been measured on or before day 5


KeyError: 'range'

### Try the same interactive technique but plot model error

this technique above will render the canopy height at any time of the season.  If we had started with a wider dataframe, that included multiple models and the error terms, this same technique could be used to render the model fits anytime during the season, as well.  Lets apply the above filtering approach, but start with a dataframe that has model errors as appended columns, instead of starting with only the raw measurements. 

In [69]:
import pandas as pd
import numpy as np
cdf = pd.read_csv('/Users/curtislisle/Dropbox/ipython-notebooks/D3M/TERRA/s4_full_height_leaf_day_v2.csv')

##### First a single XGBoost and decision tree model across the entire field

In [70]:
train_df = cdf[['day_offset','range','column','leaf_angle_alpha','leaf_angle_beta','leaf_angle_chi','leaf_angle_mean']]
target_df = cdf['canopy_height']

In [71]:
train_df.head()

,day_offset,range,column,leaf_angle_alpha,leaf_angle_beta,leaf_angle_chi,leaf_angle_mean
0,12,43,2,2.695956,1.977380,1.756464,0.435924
1,12,35,15,3.265980,2.018623,1.941012,0.396782
2,12,42,2,2.159610,1.809209,1.638744,0.471944
3,12,30,4,3.042180,2.198751,1.732985,0.444099
4,14,45,2,2.305345,1.872028,1.665387,0.462600


In [72]:
X_train = train_df.values
y_train = target_df.values
print(X_train.shape)
print(y_train.shape)

(9441, 7)
(9441,)


Below we are fitting  XGBoost, decision tree, and a linear model to the observed tuples in the "full" dataframe, which is canopy_height and leaf measurements.  We then get the results out and append them to the columns in the dataframe, so we can generate error terms and plot the results 

In [73]:
from sklearn.ensemble import GradientBoostingRegressor
gbr_mod = GradientBoostingRegressor(n_estimators=100, learning_rate=0.1, max_depth=8, random_state=0, loss='ls').fit(X_train, y_train)
pred_gbr = gbr_mod.predict(X_train)

In [74]:
# add the results columns to the input data
cdf['single_xgboost'] = pred_gbr
cdf['abserror_single_xgboost'] = 100.0*abs(cdf['canopy_height']-cdf['single_xgboost'])/cdf['canopy_height']
cdf.head()

,canopy_height,cultivar,season,range,column,leaf_angle_mean,leaf_angle_alpha,leaf_angle_beta,leaf_angle_chi,date,day_offset,single_xgboost,abserror_single_xgboost
0,15.0,PI453696,4,43,2,0.435924,2.695956,1.977380,1.756464,2017-05-13 12:00:00,12,18.350263,22.335084
1,15.0,PI145626,4,35,15,0.396782,3.265980,2.018623,1.941012,2017-05-13 12:00:00,12,15.788623,5.257488
2,19.0,PI257600,4,42,2,0.471944,2.159610,1.809209,1.638744,2017-05-13 12:00:00,12,22.092549,16.276575
3,13.0,PI569416,4,30,4,0.444099,3.042180,2.198751,1.732985,2017-05-13 12:00:00,12,17.007568,30.827446
4,17.0,PI585454,4,45,2,0.462600,2.305345,1.872028,1.665387,2017-05-15 12:00:00,14,19.366635,13.921383


In [75]:
from sklearn.tree import DecisionTreeRegressor

tree = DecisionTreeRegressor(max_depth=8).fit(X_train, y_train)
pred_tree = tree.predict(X_train)

In [76]:
# add the results columns to the input data
cdf['single_dtree'] = pred_tree
cdf['abserror_single_dtree'] = 100.0*abs(cdf['canopy_height']-cdf['single_dtree'])/cdf['canopy_height']
cdf.head()

,canopy_height,cultivar,season,range,column,leaf_angle_mean,leaf_angle_alpha,leaf_angle_beta,leaf_angle_chi,date,day_offset,single_xgboost,abserror_single_xgboost,single_dtree,abserror_single_dtree
0,15.0,PI453696,4,43,2,0.435924,2.695956,1.977380,1.756464,2017-05-13 12:00:00,12,18.350263,22.335084,20.684211,37.894737
1,15.0,PI145626,4,35,15,0.396782,3.265980,2.018623,1.941012,2017-05-13 12:00:00,12,15.788623,5.257488,16.200000,8.000000
2,19.0,PI257600,4,42,2,0.471944,2.159610,1.809209,1.638744,2017-05-13 12:00:00,12,22.092549,16.276575,20.684211,8.864266
3,13.0,PI569416,4,30,4,0.444099,3.042180,2.198751,1.732985,2017-05-13 12:00:00,12,17.007568,30.827446,16.200000,24.615385
4,17.0,PI585454,4,45,2,0.462600,2.305345,1.872028,1.665387,2017-05-15 12:00:00,14,19.366635,13.921383,20.684211,21.671827


### now the per cultivar model

In [81]:
import warnings
warnings.filterwarnings(action='ignore')


In [82]:
gbr_models = {}
predictions = {}
list_of_counts = []
count = 0
grouped = cdf.groupby(['cultivar'])
for name,group in grouped:
    #print(name)
    # pick the features to use for training
    train_df = group[['day_offset','range','column','leaf_angle_alpha','leaf_angle_beta','leaf_angle_chi','leaf_angle_mean']]
    # identify the 'target' feature to try to predict
    target_df = group['canopy_height']
    X_train = train_df.values
    y_train = target_df.values
    # record how many points were used for training
    countRec = {'cultivar': name, 'count': X_train.shape[0]}
    list_of_counts.append(countRec)
    # train a model for this cultivar in this location and store the trained model in a dictionary
    gbr_models[name] = GradientBoostingRegressor(n_estimators=100, learning_rate=0.1, max_depth=8, random_state=0, loss='ls').fit(X_train, y_train)
    gbr_pred = gbr_models[name].predict(X_train)
    count += 1
    # add the model results back into the dataframe so we can plot the actual and predicted against all the indepedent variables
    train_df['per_cultivar_gboost'] = gbr_pred
    #put the actual target value back in the dataframe so we can plot results
    train_df['canopy_height'] = target_df
    # store the predicted results in the same dictionary organization and the trained models
    predictions[name] = train_df
    if (count % 50) == 0:
        print('in process:',count, 'models')
print('finished generating',count,'models')

in process: 50 models
in process: 100 models
in process: 150 models
in process: 200 models
in process: 250 models
in process: 300 models
in process: 350 models
finished generating 351 models


A separate model was run for each cultivar, so the output 'predictions' is a dictionary with the cultivar as the keys and all the measurements and predictions as separate dataframes. First combine the multiple cultivar predictions into a single output dataframe. Then, we can join with the main dataframe to add this model. 

In [95]:
firstTime = True

# go through each cultivar
for key in predictions.keys():
    this_df = predictions[key]
    # add cultivar name to measurements dataframe
    this_df['cultivar'] = key
    # now add these lines to the output
    if firstTime:
        per_cultivar_df = this_df
        firstTime = False
    else:
        per_cultivar_df = per_cultivar_df.append(this_df,ignore_index=True)
    
    

In [99]:
# so here is the combined output of the per cultivar model stored as a single dataframe
per_cultivar_df.sample(5)

,day_offset,range,column,leaf_angle_alpha,leaf_angle_beta,leaf_angle_chi,leaf_angle_mean,per_cultivar_gboost,canopy_height,cultivar
690,14,42,4,3.454273,2.216681,1.853789,0.418852,21.005228,21.0,PI154844
5804,19,48,6,3.166447,2.102648,1.831454,0.423150,30.004505,30.0,PI563020
1178,33,39,12,3.403513,2.136490,1.915210,0.399203,85.003718,85.0,PI156871
2527,38,35,7,4.702476,2.121827,2.306110,0.347102,122.003055,122.0,PI329351
6256,65,46,14,3.054758,2.175530,1.768477,0.430558,185.999419,186.0,PI564163


In [103]:
# calculate the error for this model and add to the dataframe
per_cultivar_df['abserror_per_cultivar_gboost'] = 100.0*abs(per_cultivar_df['canopy_height']-per_cultivar_df['per_cultivar_gboost'])/per_cultivar_df['canopy_height']
per_cultivar_df.sample(5)


,day_offset,range,column,leaf_angle_alpha,leaf_angle_beta,leaf_angle_chi,leaf_angle_mean,per_cultivar_gboost,canopy_height,cultivar,abserror_per_cultivar_gboost
9431,19,9,14,4.084993,2.389530,1.942445,0.394678,30.002093,30.0,Unknown off type,0.006978
4373,56,15,15,3.040167,1.846235,1.976738,0.406046,243.999973,244.0,PI506030,0.000011
6449,32,49,9,3.262447,2.097850,1.892783,0.403903,84.003724,84.0,PI569264,0.004434
2504,117,44,7,1.106452,1.202969,1.542557,0.509578,370.995042,371.0,PI329338,0.001336
3612,56,47,10,2.109962,1.518904,1.846348,0.444007,232.000050,232.0,PI330169,0.000022


Now, we use the range,column, and cultivar type to merge the per-cultivar model with the original models and measurements.  Our result is a single dataframe that has the measurements, predictions, and errors of three different models combined.  We can use this dataframe as source data for rendering.  Before merging, we strip down the per-cultivar model to remove duplicate columns, so the columns don't become renamed by PANDAS because they are duplicates.  If we didn't do this, the result would include canopy_height_x and canopy_height_y, etc. 

In [121]:
reduced_per_cultivar_df = per_cultivar_df[['cultivar','range','column','per_cultivar_gboost','abserror_per_cultivar_gboost','day_offset']]
combine_df = pd.merge(cdf,reduced_per_cultivar_df,on=['range','column','cultivar','day_offset'])
combine_df.sample(5)

,canopy_height,cultivar,season,range,column,leaf_angle_mean,leaf_angle_alpha,leaf_angle_beta,leaf_angle_chi,date,day_offset,single_xgboost,abserror_single_xgboost,single_dtree,abserror_single_dtree,per_cultivar_gboost,abserror_per_cultivar_gboost
3381,130.0,PI329256,4,5,10,0.438429,2.386953,1.711349,1.830542,2017-06-10 12:00:00,40,137.807250,6.005577,146.823529,12.941176,130.002408,0.001852
6415,139.0,PI569422,4,47,2,0.378292,3.520821,1.871137,2.136024,2017-06-16 12:00:00,46,169.054658,21.622056,183.476190,31.997259,139.000914,0.000657
7752,229.0,PI563350,4,9,8,0.446677,2.597361,2.036907,1.699352,2017-07-01 12:00:00,61,238.811610,4.284546,246.661417,7.712409,228.999278,0.000315
9231,280.0,PI453696,4,26,13,0.577065,1.107188,1.529483,1.232676,2017-07-08 12:00:00,68,276.800018,1.142851,273.623229,2.277418,279.997772,0.000796
2252,109.0,PI585966,4,32,13,0.392432,3.688123,2.192242,1.957536,2017-06-06 12:00:00,36,108.573143,0.391612,112.364964,3.087122,109.002623,0.002406


In [122]:
combine_df.sample(10)

,canopy_height,cultivar,season,range,column,leaf_angle_mean,leaf_angle_alpha,leaf_angle_beta,leaf_angle_chi,date,day_offset,single_xgboost,abserror_single_xgboost,single_dtree,abserror_single_dtree,per_cultivar_gboost,abserror_per_cultivar_gboost
5101,199.0,PI562970,4,37,8,0.507344,1.652908,1.650230,1.503082,2017-07-11 12:00:00,71,247.152231,24.197101,273.623229,37.499110,198.999265,0.000370
6263,274.0,PI156871,4,3,15,0.399228,3.017404,1.859023,1.967877,2017-06-29 12:00:00,59,270.513580,1.272416,240.298013,12.299995,273.998785,0.000443
2666,103.0,PI570106,4,24,10,0.333235,4.895106,2.327788,2.236247,2017-06-08 12:00:00,38,115.797996,12.425239,133.560000,29.669903,103.002502,0.002429
3861,179.0,PI154844,4,13,14,0.444072,2.116695,1.543702,1.836673,2017-06-15 12:00:00,45,185.729015,3.759226,178.000000,0.558659,179.000981,0.000548
3695,158.0,PI563009,4,6,10,0.391910,3.427130,2.003437,2.002132,2017-06-15 12:00:00,45,164.719607,4.252916,168.659722,6.746660,158.001319,0.000835
2118,110.0,PI63715,4,24,3,0.436621,2.498794,1.861519,1.765332,2017-06-05 12:00:00,35,99.271672,9.753025,101.326087,7.885375,110.002704,0.002458
2049,86.0,PI152728,4,44,3,0.365100,4.021094,2.192146,2.078975,2017-06-05 12:00:00,35,92.953177,8.085090,110.971963,29.037166,86.002396,0.002786
8265,307.0,PI585608,4,4,9,0.447640,2.189227,1.673803,1.758629,2017-08-15 12:00:00,106,314.393960,2.408456,330.673913,7.711372,306.997322,0.000872
4822,272.0,PI452542,4,5,11,0.438126,2.223047,1.565023,1.860020,2017-06-26 12:00:00,56,257.922690,5.175481,233.208791,14.261474,271.999266,0.000270
3474,136.0,PI329338,4,44,7,0.417255,2.506992,1.681584,1.898461,2017-06-10 12:00:00,40,143.691442,5.655472,142.333333,4.656863,136.001340,0.000985


save out this result to keep from having to rebuild it. 

In [123]:
combine_df.to_csv("s4_height_and_models.csv",index=False)

### Generalized 'selected day' plotting 
Now, lets generalize the "selected day" rendering method to allow us to pick any feature, not just canopy_height.  This is identical to the earlier cell  except for letting the feature be selectable.  This plot takes a dataframe and a selected feature name, then returns the state of the field for that feature up to the selected day.  We are passing it a dataframe with measurements plus a feature name.  This is the generalization of the method used above and this one is what we could use underneath an interface. 

In [138]:

def addPlotMarker(cultivar,rng,column,selectedFeatureName,featureValue):
    global plotlist
    mark = {}
    mark['cultivar'] = cultivar
    mark['range'] = rng
    mark['column'] = column
    mark[selectedFeatureName] = featureValue
    plotlist.append(mark)

# this method takes an input day of the season and generates an output dataframe with the most recent
#  measurement of a selectedFeature taken for each location in the field.  It is a way to watch the field develop
# over time during the season. 

def renderCanopyHeightOnDay(dataFrm, selectedDay,selectedFeature):
    global maxRange
    global maxColumn
    
    # first get rid of observations after the query day
    before_df = dataFrm.loc[dataFrm['day_offset'] <= selectedDay]
    print(before_df.shape)

    # group all the measurements so far by cultivar 
    grouped = before_df.groupby(['range','column'])

    # now loop through these by cultivar and select only the measurement with the highest day_offset value (the most recent)
    recentlist = []
    for name, group in grouped:
        #print(name)
        selected = group['day_offset'].idxmax()  # this selects the highest value index
        # the index is a lookup into the original dataframe, so put this entry in the list for plotting
        recentlist.append(dataFrm.iloc[selected])  

    # how many cultivars did we find that had a measurement on or before our day?    
    print(len(recentlist),"cultivars have been measured on or before day",selectedDay)
    recent_df = pd.DataFrame(recentlist)
    
    # now fill out the entire field by querying the values at each location from the 
    # recent dataframe and filling in a plotting list.  This global list (plotlist) needs to be empty
    # before running this algorithm.  
    
    # TODO: remove global dependency/ side effect on plotlist, maxColumn, maxRange?
   
    cultivarCount = 0
    measurementCount = 0
    # go once across the entire field by using range and column indices
    for rng in range(2,int(maxRange)):
        for col in range(2,int(maxColumn)):
            #print(rng,col)
            # find which cultivar is in this spot in the field
            CultivarListInThisSpot = dataFrm.loc[(dataFrm['range'] == rng) & (dataFrm['column']==col)]['cultivar']
            print
            # return a Series of the cultivar names. If the square isn't empty, get the cultivar name from the list. 
            # all cultivar names should be identical since we have selected multiple measurements (on different days) from the same location
            if len(CultivarListInThisSpot)> 0:
                cultivarCount += 1
                thisCultivar = CultivarListInThisSpot.values[0]
                thisMeasurement = recent_df.loc[(recent_df['range'] == rng) & (recent_df['column'] == col)][selectedFeature]
                # depending on the day, we might or might not have had a previous measurement, so check there was a measurement
                # before plotting.  This filter prevents a run-time error trying to plot non-existent measurements.  See the
                # "else" case below for when there is no previous measurement. 
                if len(thisMeasurement)>0:
                    measurementCount += 1
                    thisMeasurementValue = thisMeasurement.values[0]
                    addPlotMarker(thisCultivar,rng,col,selectedFeature,thisMeasurementValue)
                else:
                    # fill in empty entries for locations where there were no measurements. This happens more during
                    # the early part of the season because measurements haven't been taken in some locations yet. This
                    # way, the plot will always render the full field because all locations will have an entry, even
                    # if it is zero because no measurements have been taken yet.
                    addPlotMarker(thisCultivar,rng,col,selectedFeature,0.0)
             
    plotdf = pd.DataFrame(plotlist)
    print('cultivars found:',cultivarCount)
    print('measurements found:',measurementCount)
    print('plotted',len(plotlist),'values')
    return plotdf


In [143]:
import altair as alt

maxColumn = cdf.describe().loc['max','column']
maxRange = cdf.describe().loc['max','range']

# this accumulates, so should be cleared out before each rendering
plotlist = []
selectedDay = 15
plotFeature = 'abserror_per_cultivar_gboost'
#plotFeature = 'leaf_angle_mean'
plotdf = renderCanopyHeightOnDay(combine_df,selectedDay,plotFeature)


charttitle = 'Season4 '+ plotFeature+': Day '+str(selectedDay)
alt.Chart(plotdf,title=charttitle).mark_rect().encode(
x='column:O',
y='range:O',
color=plotFeature+':Q',
tooltip=[
    alt.Tooltip('cultivar:O', title='Cultivar'),
    alt.Tooltip(plotFeature+':Q', title=plotFeature),
    alt.Tooltip('range:O',title='range'),
    alt.Tooltip('column:O',title='column')
]
)

(71, 17)
61 cultivars have been measured on or before day 15
cultivars found: 703
measurements found: 57
plotted 703 values


alt.Chart(...)

### Merging in the per-location model
so we have single model, per-cultivar model, and per-location model all in a single dataframe.  This dataframe can be rendered by Trelliscope (hopefully). 

In [4]:
import pandas as pd
full_df = pd.read_csv('s4_height_and_models.csv')
per_location_df = pd.read_csv('s4_per_location_gboost.csv')

Now, we use the day,range,column, and cultivar type to merge the per-location model with the original models and measurements.  Our result is a single dataframe that has the measurements, predictions, and errors of all different models combined.  We can use this dataframe as source data for rendering.  Before merging, we strip down the per-cultivar model to remove duplicate columns, so the columns don't become renamed by PANDAS because they are duplicates.  If we didn't do this, the result would include canopy_height_x and canopy_height_y, etc. 

In [5]:
reduced_per_location_df = per_location_df[['cultivar','range','column','per_location_gboost','abserror_per_location_gboost','day_offset']]
combine_df = pd.merge(full_df,reduced_per_location_df,on=['range','column','cultivar','day_offset'])
combine_df.sample(5)

,canopy_height,cultivar,season,range,column,leaf_angle_mean,leaf_angle_alpha,leaf_angle_beta,leaf_angle_chi,date,day_offset,single_xgboost,abserror_single_xgboost,single_dtree,abserror_single_dtree,per_cultivar_gboost,abserror_per_cultivar_gboost,per_location_gboost,abserror_per_location_gboost
8302,342.0,PI570145,4,8,8,0.475153,1.933667,1.677459,1.624290,2017-08-15 12:00:00,106,341.285386,0.208951,330.673913,3.311721,341.997101,0.000848,341.998626,0.000402
5486,327.0,PI329333,4,26,10,0.445410,2.092306,1.581673,1.789369,2017-08-10 12:00:00,101,324.425808,0.787215,330.673913,1.123521,326.996769,0.000988,326.997409,0.000792
5471,267.0,PI641821,4,10,5,0.516428,1.250168,1.358091,1.497582,2017-08-10 12:00:00,101,287.185437,7.560089,280.400000,5.018727,266.998386,0.000604,266.998960,0.000389
4531,340.0,PI329435,4,46,11,0.468164,1.527611,1.323635,1.737598,2017-08-29 12:00:00,120,333.980664,1.770393,335.944444,1.192810,339.995453,0.001337,339.995553,0.001308
1270,84.0,PI330833,4,49,11,0.371634,3.680206,2.065180,2.058655,2017-06-03 12:00:00,33,86.648740,3.153261,94.569444,12.582672,84.002752,0.003276,84.002949,0.003511


In [6]:
# just checking that we had the same entries and that the two extra columns merged in correctly
print(full_df.shape)
print(combine_df.shape)

(9441, 17)
(9441, 19)


In [7]:
combine_df.to_csv('s4_final_heights_and_models.csv')

Now we have written out this dataframe with all the models and errors as extra columns.  We will use this dataset for Trelliscope and for exploring other arbor_nova apps.

### all-in-one for on-demand model generation
Below, develop a single cell that can be adapted into an arbor_nova method to all the scientists to set hyperparameters and run models on demand.

In [16]:
import pandas as pd
import numpy as np
from sklearn.ensemble import GradientBoostingRegressor

# define a method that receives a dataframe, runs a model, and appends columns
# containing the model values and errors to the original dataframe and returns the resulting
# dataframe for further processing into a visualization

def runXGBoostPerCultivarOnSeasonMeasurements(dataFrm,estimators=100,learningRate=0.1,maxDepth=8):
    gbr_models = {}
    predictions = {}
    list_of_counts = []
    count = 0
    grouped = dataFrm.groupby(['cultivar'])
    for name,group in grouped:
        #print(name)
        # pick the features to use for training
        train_df = group[['day_offset','range','column','leaf_angle_alpha','leaf_angle_beta','leaf_angle_chi','leaf_angle_mean']]
        # identify the 'target' feature to try to predict
        target_df = group['canopy_height']
        X_train = train_df.values
        y_train = target_df.values
        # record how many points were used for training
        countRec = {'cultivar': name, 'count': X_train.shape[0]}
        list_of_counts.append(countRec)
        # train a model for this cultivar in this location and store the trained model in a dictionary
        gbr_models[name] = GradientBoostingRegressor(
                                n_estimators=estimators, 
                                learning_rate=learningRate, 
                                max_depth=maxDepth,
                                random_state=0, loss='ls'
                                                ).fit(X_train, y_train)
        gbr_pred = gbr_models[name].predict(X_train)
        count += 1
        # add the model results back into the dataframe so we can plot the actual and predicted against all the indepedent variables
        train_df['per_cultivar_gboost'] = gbr_pred
  
        #put the actual target value back in the dataframe so we can plot results
        train_df['canopy_height'] = target_df
        
        # calculate the per measurement error
        train_df['abserror_per_cultivar_gboost'] = 100.0*abs(train_df['canopy_height']-train_df['per_cultivar_gboost'])/train_df['canopy_height']

        # store the predicted results in the same dictionary organization and the trained models
        predictions[name] = train_df
        if (count % 50) == 0:
            print('in process:',count, 'models')
    print('finished generating',count,'models')
    
    # A separate model was run for each cultivar, so the output 'predictions' is a dictionary 
    # with the cultivar as the keys and all the measurements and predictions as separate 
    # dataframes. First combine the multiple cultivar predictions into a single output
    # dataframe. Then, we can join with the main dataframe to add this model.   
    firstTime = True
    # go through each cultivar
    for key in predictions.keys():
        this_df = predictions[key]
        # add cultivar name to measurements dataframe
        this_df['cultivar'] = key
        # now add these lines to the output
        if firstTime:
            per_cultivar_df = this_df
            firstTime = False
        else:
            per_cultivar_df = per_cultivar_df.append(this_df,ignore_index=True)
    print('per_cultivar_df',per_cultivar_df)
    return per_cultivar_df


def addPlotMarker(plotlist,cultivar,rng,column,selectedFeatureName,featureValue,day):
    mark = {}
    mark['cultivar'] = cultivar
    mark['range'] = rng
    mark['column'] = column
    mark['day'] = day
    mark[selectedFeatureName] = featureValue
    plotlist.append(mark)

# this method takes an input day of the season and generates an output dataframe with the most recent
#  measurement of a selectedFeature taken for each location in the field.  It is a way to watch the field develop
# over time during the season. 

def renderFeatureOnDay(dataFrm, selectedDay,selectedFeature):
    summary_df = dataFrm.describe()
    minColumn = summary_df.loc['min','column']
    minRange =  summary_df.loc['min','range']
    maxColumn = summary_df.loc['max','column']
    maxRange =  summary_df.loc['max','range']

    # first get rid of observations after the query day
    before_df = dataFrm.loc[dataFrm['day_offset'] <= selectedDay]
    print(before_df.shape)

    # group all the measurements so far by cultivar 
    grouped = before_df.groupby(['range','column'])

    # now loop through these by cultivar and select only the measurement with the highest day_offset value (the most recent)
    recentlist = []
    for name, group in grouped:
        #print(name)
        selected = group['day_offset'].idxmax()  # this selects the highest value index
        # the index is a lookup into the original dataframe, so put this entry in the list for plotting
        recentlist.append(dataFrm.iloc[selected])  

    # how many cultivars did we find that had a measurement on or before our day?    
    print(len(recentlist),"cultivars have been measured on or before day",selectedDay)
    recent_df = pd.DataFrame(recentlist)
    
    # now fill out the entire field by querying the values at each location from the 
    # recent dataframe and filling in a plotting list.  This global list (plotlist) needs to be empty
    # before running this algorithm.  
   
    plotlist = []
    cultivarCount = 0
    measurementCount = 0
    # go once across the entire field by using range and column indices
    for rng in range(int(minRange),int(maxRange)):
        for col in range(int(minColumn),int(maxColumn)):
            # find which cultivar is in this spot in the field
            CultivarListInThisSpot = dataFrm.loc[(dataFrm['range'] == rng) & (dataFrm['column']==col)]['cultivar']
            print
            # return a Series of the cultivar names. If the square isn't empty, get the cultivar name from the list. 
            # all cultivar names should be identical since we have selected multiple measurements (on different days) from the same location
            if len(CultivarListInThisSpot)> 0:
                cultivarCount += 1
                thisCultivar = CultivarListInThisSpot.values[0]
                # catch exception because recent_df might be empty if day requested is before all measurments
                try:
                    thisMeasurement = recent_df.loc[(recent_df['range'] == rng) & (recent_df['column'] == col)][selectedFeature]
                    thisMeasurementDay = recent_df.loc[(recent_df['range'] == rng) & (recent_df['column'] == col)]['day_offset']
       
                    # depending on the day, we might or might not have had a previous measurement, so check there was a measurement
                    # before plotting.  This filter prevents a run-time error trying to plot non-existent measurements.  See the
                    # "else" case below for when there is no previous measurement. 
                    if len(thisMeasurement)>0:
                        measurementCount += 1
                        thisMeasurementValue = thisMeasurement.values[0]
                        thisMeasurementDayValue = thisMeasurementDay.values[0]
                        addPlotMarker(plotlist,thisCultivar,rng,col,selectedFeature,thisMeasurementValue,thisMeasurementDayValue)
                    else:
                        # fill in empty entries for locations where there were no measurements. This happens more during
                        # the early part of the season because measurements haven't been taken in some locations yet. This
                        # way, the plot will always render the full field because all locations will have an entry, even
                        # if it is zero because no measurements have been taken yet.
                        addPlotMarker(plotlist,thisCultivar,rng,col,selectedFeature,0.0,0)
                except:
                    # support the case where the date was so low, there were no measurements at all
                    addPlotMarker(plotlist,thisCultivar,rng,col,selectedFeature,0.0,0)
             
    plotdf = pd.DataFrame(plotlist)
    #print('cultivars found:',cultivarCount)
    #print('measurements found:',measurementCount)
    #print('plotted',len(plotlist),'values')
    return plotdf


import warnings
warnings.filterwarnings(action='ignore')

# delcare all the parameters that would be specified through the interface
depth = 8
estimators = 100
learn = 0.1
day = 50
#feature = 'canopy_height'
#feature = 'leaf_angle_beta'
feature = 'abserror_per_cultivar_gboost'

test_df = pd.read_csv('s4_full_height_leaf_day_v2.csv')

predict_df = runXGBoostPerCultivarOnSeasonMeasurements(
            test_df,
            estimators=estimators,
            maxDepth=depth,
            learningRate=learn)
result_df = renderFeatureOnDay(predict_df, day,feature)
result_df.head()

in process: 50 models
in process: 100 models
in process: 150 models
in process: 200 models
in process: 250 models
in process: 300 models
in process: 350 models
finished generating 351 models
per_cultivar_df       day_offset  range  column  leaf_angle_alpha  leaf_angle_beta  \
0             14     54       6          2.862611         2.032145   
1             15     54       8          3.187505         2.155901   
2             18     54      12          2.613957         1.892850   
3             19     54      12          1.857061         1.592292   
4             23     54      12          1.252334         1.418767   
...          ...    ...     ...               ...              ...   
9436          36     32      10          4.288913         2.309921   
9437          38     32      10          4.438680         2.298816   
9438          39     32      10          3.506227         1.948681   
9439          40     32      10          3.036119         1.831445   
9440          45      9

,cultivar,range,column,day,abserror_per_cultivar_gboost
0,Big_Kahuna,2,14,34,0.001884
1,SP1615,2,15,34,0.002201
2,PI329465,3,2,36,0.002324
3,PI22913,3,3,40,0.000977
4,PI145626,3,4,47,0.000142
